In [ ]:
%cd ..

# Load Data


## Load Raw

In [ ]:
import os 
import pandas as pd

RecName = 'Smoking'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


In [ ]:

# load all records
df = pd.read_csv(raw_file_path)
df

## Select Columns 

In [ ]:
parent_ids = ['PID','ECID']
focal_ids = ['RID']


selected_cols = ['V', 'DT']

df = df[parent_ids + focal_ids + selected_cols]
df

## Basic Filtering 

In [ ]:
# nothing

## DF Rec Raw

In [ ]:
df_RecRaw = df
df_RecRaw

# Fields in Records

In [ ]:
df = df_RecRaw.copy()

## Field: ID

In [ ]:
RecName = 'Smoking'
RID = RecName + 'ID'
df = df.rename(columns = {'RID': RID})
assert df[RID].value_counts().max() == 1
df.head()

## Field: V

In [ ]:
df['V']

## Field: DT

In [ ]:
df['DT']

## Call Parents

In [ ]:
def load_df_data_from_folder(data_folder, IDName):
    file_list = os.listdir(data_folder)
    # file_lis
    df = pd.concat([pd.read_pickle(os.path.join(data_folder, i)) for i in file_list if '.p' in i]).reset_index(drop = True)
    df = df.sort_values(IDName).reset_index(drop = True)
    return df# .shape

fullrec_folder = 'data/ProcData/RecFolder/EC'
df_Pat = load_df_data_from_folder(fullrec_folder, 'ECID')[['ECID','PID']]
print(df_Pat.shape)
df_Pat.head()
print(df_Pat)


df = pd.merge(df_Pat, df_RecRaw, on = parent_ids, how = 'left')
df

In [ ]:
import numpy as np

d = df
for col in focal_ids: 
    d.loc[d[col].isnull(), col] = ['Missing' + str(i) for i in np.ones(d[col].isnull().sum()).cumsum()]
    # d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

## Pin Down

In [ ]:
df_proc = df[['PID', 'ECID', RID, 'V','DT']]
df_proc

# Save and Load

## Save

In [ ]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
RANGE_SIZE = 10000; IDNAME = 'PID'

In [ ]:
# the folder to save the Record.
###############
RecName = RecName
df = df_proc
###############

rec_folder = 'Data/ProcData/RecFolder'
data_folder = os.path.join(rec_folder, RecName)
print(data_folder, '<----- fullrec_folder to save the records')

if not os.path.exists(data_folder): 
    os.makedirs(data_folder)

write_df_to_folders(RecName, data_folder, df, IDNAME)

## Load

In [ ]:
from recfldgrn.datapoint import load_df_data_with_RecName
df = load_df_data_with_RecName(rec_folder, RecName)
print(df.shape)
df.head()

## Load One Sample

In [ ]:
from recfldgrn.datapoint import DataPoint
####################
PIDValue = 'P5'  
######################
Pat = DataPoint(IDNAME, PIDValue, rec_folder, RANGE_SIZE)
Pat

In [ ]:
Pat.get_df_rec(RecName)